<a href="https://colab.research.google.com/github/JimKing100/nfl-test/blob/master/predictions/Predictions_Rookies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Installs
%%capture
!pip install category_encoders==2.0.0

In [0]:
# Import libraries
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
import category_encoders as ce
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
from xgboost import XGBRegressor


import category_encoders as ce

In [0]:
player_df = pd.read_csv('https://raw.githubusercontent.com/JimKing100/nfl-test/master/data-actuals/players_full.csv')
kickers_df = pd.read_csv('https://raw.githubusercontent.com/JimKing100/nfl-test/master/data-actuals/actuals_rookie_kickers.csv')
offense_df = pd.read_csv('https://raw.githubusercontent.com/JimKing100/nfl-test/master/data-actuals/actuals_rookie_offense.csv')
kickers2019_df = pd.read_csv('https://raw.githubusercontent.com/JimKing100/nfl-test/master/data-actuals/actuals_rookie2019_kickers.csv')
offense2019_df = pd.read_csv('https://raw.githubusercontent.com/JimKing100/nfl-test/master/data-actuals/actuals_rookie2019_offense.csv')
rookies_df = pd.read_csv('https://raw.githubusercontent.com/JimKing100/nfl-test/master/data-revised/rookies.csv')
actuals_df = pd.concat([kickers_df, offense_df], ignore_index=True)
actuals2019_df = pd.concat([kickers2019_df, offense2019_df], ignore_index=True)

In [4]:
player_df.head()

,player,first,last,name,position1,start,cteam,height,weight,dob,forty,bench,vertical,broad,shuttle,cone,arm,hand,dpos,col,dv,jnum,dcp
0,SF,San Francisco,49ers,San Francisco 49ers,DF,2000,SF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CHI,Chicago,Bears,Chicago Bears,DF,2000,CHI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CIN,Cincinnati,Bengals,Cincinnati Bengals,DF,2000,CIN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BUF,Buffalo,Bills,Buffalo Bills,DF,2000,BUF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,DEN,Denver,Broncos,Denver Broncos,DF,2000,DEN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
actuals_df.head()

,player,first,last,name,position1,game1,game2,game3,game4,game5,game6,game7,game8,game9,game10,game11,game12,game13,game14,game15,game16
0,AV-0400,Adam,Vinatieri,Adam Vinatieri,K,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,MB-4600,Matt,Bryant,Matt Bryant,K,7.0,14.0,10.0,1.0,3.0,4.0,3.0,6.0,-1.0,13.0,2.0,10.0,10.0,12.0,8.0,4.0
2,RG-1500,Robbie,Gould,Robbie Gould,K,5.0,4.0,4.0,7.0,7.0,5.0,6.0,7.0,13.0,2.0,10.0,7.0,3.0,3.0,0.0,0.0
3,MN-0800,Mike,Nugent,Mike Nugent,K,0.0,6.0,8.0,3.0,0.0,6.0,2.0,8.0,2.0,15.0,3.0,13.0,10.0,3.0,13.0,0.0
4,SG-0800,Stephen,Gostkowski,Stephen Gostkowski,K,5.0,5.0,0.0,7.0,8.0,4.0,8.0,9.0,5.0,5.0,7.0,8.0,16.0,7.0,10.0,13.0


In [6]:
rookies_df.head()

,player,first,last,name,position1,start
0,MG-0306,Matt,Gay,Matt Gay,K,2019
1,CM-2137,Chase,McLaughlin,Chase McLaughlin,K,2019
2,AS-0575,Austin,Seibert,Austin Seibert,K,2019
3,JS-3825,Joey,Slye,Joey Slye,K,2019
4,DB-2750,David,Blough,David Blough,QB,2019


In [7]:
actuals2019_df.head()

,player,first,last,name,position1,game1,game2,game3,game4,game5,game6,game7,game8,game9,game10,game11,game12,game13,game14,game15,game16
0,MG-0306,Matt,Gay,Matt Gay,K,5.0,8.0,15.0,15.0,7.0,6.0,10.0,12.0,13.0,7.0,5.0,8.0,9.0,9.0,10.00,1.00
1,CM-2137,Chase,McLaughlin,Chase McLaughlin,K,8.0,5.0,10.0,5.0,11.0,6.0,14.0,10.0,1.0,7.0,10.0,0.0,0.0,0.0,0.00,0.00
2,AS-0575,Austin,Seibert,Austin Seibert,K,1.0,13.0,7.0,11.0,3.0,2.0,8.0,13.0,8.0,3.0,10.0,6.0,11.0,7.0,3.00,6.00
3,JS-3825,Joey,Slye,Joey Slye,K,11.0,16.0,8.0,13.0,2.0,14.0,4.0,6.0,5.0,3.0,10.0,2.0,10.0,8.0,8.00,4.00
4,DB-2750,David,Blough,David Blough,QB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.3,8.2,8.3,8.68,11.88


In [0]:
# The main code for iterating through the player list, calculating the points and adding the rows
# to the final_df dataframe.
def fill_df(p_df, tf):
    column_names = ['player',
                    'position1',
                    'height',
                    'weight',
                    'forty',
                    'bench',
                    'vertical',
                    'broad',
                    'shuttle',
                    'cone',
                    'arm',
                    'hand',
                    'dpos',
                    'col',
                    'dv',
                    'points' 
                    ]

    player_list = p_df['player'].tolist()

    df = pd.DataFrame(columns = column_names)

    for player in player_list:

        position1 = player_df['position1'].loc[(player_df['player']==player)].iloc[0]
        height = player_df['height'].loc[(player_df['player']==player)].iloc[0]
        weight = player_df['weight'].loc[(player_df['player']==player)].iloc[0]
        forty = player_df['forty'].loc[(player_df['player']==player)].iloc[0]
        bench = player_df['bench'].loc[(player_df['player']==player)].iloc[0]
        vertical = player_df['vertical'].loc[(player_df['player']==player)].iloc[0]
        broad = player_df['broad'].loc[(player_df['player']==player)].iloc[0]
        shuttle = player_df['shuttle'].loc[(player_df['player']==player)].iloc[0]
        core = player_df['cone'].loc[(player_df['player']==player)].iloc[0]
        arm = player_df['arm'].loc[(player_df['player']==player)].iloc[0]
        hand = player_df['hand'].loc[(player_df['player']==player)].iloc[0]
        dpos = player_df['dpos'].loc[(player_df['player']==player)].iloc[0]
        college = player_df['col'].loc[(player_df['player']==player)].iloc[0]
        division = player_df['dv'].loc[(player_df['player']==player)].iloc[0]
        if tf == 'train':
            points = actuals_df.loc[(actuals_df['player']==player)].iloc[0, 5:21].sum()
        else:
            points = actuals2019_df.loc[(actuals2019_df['player']==player)].iloc[0, 5:21].sum()

        df = df.append({'player': player,
                        'position1': position1,
                        'height': height,
                        'weight': weight,
                        'forty': forty,
                        'bench': bench,
                        'vertical': vertical,
                        'broad': broad,
                        'shuttle': shuttle,
                        'cone': core,
                        'arm': arm,
                        'hand': hand,
                        'dpos': dpos,
                        'col': college,
                        'dv': division,
                        'points': points 
                        }, ignore_index=True)
        
    return df

In [0]:
train_df = fill_df(actuals_df, 'train')
test_df = fill_df(rookies_df, 'final')

In [10]:
train_df.tail()

,player,position1,height,weight,forty,bench,vertical,broad,shuttle,cone,arm,hand,dpos,col,dv,points
510,SM-2825,WR,70.0,189.0,4.56,13.0,33.5,121.0,4.40,6.75,0.000,0.000,0.0,USC,Pacific 12,0.0
511,TQ-0100,WR,71.0,203.0,4.46,17.0,33.5,104.0,4.17,6.91,30.750,10.125,256.0,SMU,American Athletic,13.5
512,TS-1425,WR,72.0,200.0,4.48,19.0,32.5,125.0,4.20,6.90,0.000,0.000,0.0,Vanderbilt,Southeastern (SEC),27.0
513,TS-2425,WR,74.0,203.0,4.49,12.0,37.5,130.0,4.50,6.97,33.375,9.500,91.0,Central Florida,American Athletic,74.7
514,VS-0175,WR,75.0,195.0,4.38,0.0,39.5,130.0,4.27,6.89,0.000,0.000,0.0,Limestone,0,15.1


In [11]:
test_df.tail()

,player,position1,height,weight,forty,bench,vertical,broad,shuttle,cone,arm,hand,dpos,col,dv,points
101,SS-0650,WR,68.0,176.0,4.64,13.0,36.5,121.0,4.34,6.71,29.250,9.250,0.0,Illinois State,Missouri Valley,0.0
102,SS-1537,WR,69.0,181.0,4.56,11.0,31.5,115.0,4.43,7.20,29.375,8.625,0.0,Kansas,Big 12,77.5
103,TF-1450,WR,74.0,215.0,4.55,15.0,36.5,126.0,4.22,6.84,33.625,9.500,184.0,Old Dominion,Conference USA (C-USA),0.0
104,TM-2275,WR,72.0,208.0,4.35,18.0,37.5,125.0,4.15,7.01,31.500,9.125,76.0,Ohio State,Big Ten,137.9
105,VB-0325,WR,75.0,198.0,4.61,12.0,34.5,121.0,4.25,6.97,33.125,9.750,0.0,Temple,American Athletic,7.5


In [0]:
target = 'points'
X_train = train_df.drop(columns=[target])
y_train = train_df[target]
X_test = test_df.drop(columns=[target])
y_test = test_df[target]

In [0]:
# Split the initial train features and labels 80/20 into train and validate
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, train_size = 0.80, test_size = 0.20)

In [14]:
pipeline = make_pipeline(
    ce.OneHotEncoder(use_cat_names=True), 
    XGBRegressor(n_estimators=200, n_jobs=-1)
)

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_val)

# Print metrics for validation 
val_mse = mean_squared_error(y_val, y_pred)
val_rmse = np.sqrt(val_mse)
val_mae = mean_absolute_error(y_val, y_pred)
val_r2 = r2_score(y_val, y_pred)
print('Validation Mean Absolute Error:', val_mae)
print('Validation R^2:', val_r2)
print('\n')

ty_pred = pipeline.predict(X_test)

# Print metrics for test
test_mse = mean_squared_error(y_test, ty_pred)
test_rmse = np.sqrt(test_mse)
test_mae = mean_absolute_error(y_test, ty_pred)
test_r2 = r2_score(y_test, ty_pred)
print('Test Mean Absolute Error:', test_mae)
print('Test R^2:', test_r2)

[15:52:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Validation Mean Absolute Error: 36.07014687769621
Validation R^2: 0.22983390645590385


Test Mean Absolute Error: 32.70727375471367
Test R^2: 0.3710756132458516


In [19]:
final_df = test_df
# The final_df dataframe
final_df['prediction'] = ty_pred
final_df.head(50)

,player,position1,height,weight,forty,bench,vertical,broad,shuttle,cone,arm,hand,dpos,col,dv,points,prediction
0,MG-0306,K,72.0,232.0,5.20,0.0,0.0,0.0,0.00,0.00,30.500,9.000,145.0,Utah,Pacific 12,140.00,47.157749
1,CM-2137,K,73.0,190.0,4.97,0.0,0.0,0.0,0.00,0.00,29.625,9.250,0.0,Illinois,Big Ten,87.00,31.719452
2,AS-0575,K,69.0,209.0,5.14,0.0,0.0,0.0,0.00,0.00,30.375,8.875,170.0,Oklahoma,Big 12,112.00,69.943565
3,JS-3825,K,71.0,210.0,0.00,0.0,0.0,0.0,0.00,0.00,0.000,0.000,0.0,Virginia Tech,Atlantic Coast (ACC),124.00,55.463665
4,DB-2750,QB,72.0,205.0,4.91,0.0,31.5,107.0,4.55,7.22,29.625,9.375,0.0,Purdue,Big Ten,55.36,-5.486490
5,DH-1912,QB,75.0,231.0,5.04,0.0,28.5,0.0,0.00,0.00,33.500,9.625,15.0,Ohio State,Big Ten,79.70,100.737839
6,DH-3450,QB,73.0,211.0,4.79,0.0,32.0,103.0,4.53,7.38,30.125,9.250,0.0,Samford,Southern,52.32,10.996941
7,DJ-2275,QB,77.0,221.0,4.72,0.0,33.5,120.0,4.41,7.00,32.500,9.750,6.0,Duke,Atlantic Coast (ACC),217.98,143.273483
8,DL-1787,QB,76.0,228.0,4.69,0.0,31.0,112.0,4.12,7.03,32.500,9.000,42.0,Missouri,Southeastern (SEC),68.00,77.191322
9,GM-1350,QB,73.0,225.0,4.97,0.0,33.5,116.0,4.45,7.14,31.750,10.125,178.0,Washington State,Pacific 12,232.24,34.798237
